In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your AnimalShelter class from your Python module
from Animal_Shelter import AnimalShelter

# Data Manipulation / Model
username = "aacuser"
password = "SNHU1234"

# Connect to the database via the CRUD Module
db = AnimalShelter(username, password)

# Read data from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column to avoid issues with data_table
df.drop(columns=['_id'], inplace=True)

# Create the Dash application
app = JupyterDash(__name__)

# Load and encode the logo image
image_filename = 'my-image.png'  # Replace with your own image file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard Layout / View
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width=200),
        html.Hr(),
        html.Div([
            # Add radio buttons for reset and filtering
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Reset', 'value': 'reset'},
                    {'label': 'Water Rescue', 'value': 'filter'},
                    {'label': 'Mountain or Wilderness', 'value': 'mountain_wilderness'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'disaster_individual_tracking'},
                ],
                value='reset'
            ),
        ]),
        html.Hr(),
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True}
                for i in df.columns
            ],
            data=df.to_dict('records'),
            page_size=10,
            editable=True,
            sort_action="native",
            row_deletable=True,
            sort_mode="multi",
            selected_columns=[],
            selected_rows=[0],
            page_action="native",
            page_current=0,
            hidden_columns=[],
            filter_action="native",
        ),
        html.Br(),
        html.Hr(),
        html.H2("Malcolm McGee SNHU 340"),
        html.Div(
            className='row',
            style={'display': 'flex'},
            children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                )
            ]
        )
    ])
])

# Callback to update the chart based on filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]  # Add the filter-type input
)
def update_graphs(viewData, filter_type):
    if viewData is None:
        return []  # No data to display

    dff = pd.DataFrame.from_dict(viewData)

    if filter_type == 'filter':
        # Filter for water rescue dogs
        filtered_dff = dff[
            (dff['breed'].str.contains('Labrador Retriever Mix|Chesapeake Bay Retriever|Newfoundland')) & 
            (dff['sex_upon_outcome'] == 'Intact Female') & 
            (dff['age_upon_outcome_in_weeks'].between(26, 156))
        ]
        title = 'Water Rescue'
    elif filter_type == 'mountain_wilderness':
        # Filter for mountain or wilderness dogs
        filtered_dff = dff[
            (dff['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler')) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'].between(26, 156))
        ]
        title = 'Mountain or Wilderness'
    elif filter_type == 'disaster_individual_tracking':
        # Filter for disaster or individual tracking dogs
        filtered_dff = dff[
            (dff['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler')) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'].between(20, 300))
        ]
        title = 'Disaster or Individual Tracking'
    else:
        # No filter or reset
        filtered_dff = dff
        title = 'Preferred Animals'

    # Create a pie chart based on your data
    figure = px.pie(filtered_dff, names='breed', title=title)

    return [dcc.Graph(figure=figure)]

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    if filter_type == 'filter':
        # Filter for water rescue dogs
        filtered_data = df[
            (df['breed'].str.contains('Labrador Retriever Mix|Chesapeake Bay Retriever|Newfoundland')) & 
            (df['sex_upon_outcome'] == 'Intact Female') & 
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif filter_type == 'mountain_wilderness':
        # Filter for mountain or wilderness dogs
        filtered_data = df[
            (df['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler')) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif filter_type == 'disaster_individual_tracking':
        # Filter for disaster or individual tracking dogs
        filtered_data = df[
            (df['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler')) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(20, 300))
        ]
    else:
        # No filter or reset
        filtered_data = df

    return filtered_data.to_dict('records')

# Callback to highlight selected rows in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if viewData is None or not viewData:
        return dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id='base-layer-id')
            ]
        )

    if index is None or not index:
        row = 0
    else:
        row = index[0]

    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[30.75, -97.48],
        zoom=10,
        children=[
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                      children=[
                          dl.Tooltip(dff.iloc[row, 4]),
                          html.H1('Animal Name'),
                          html.P(dff.iloc[row, 9])
                      ])
        ])

# Run the application
app.run_server(debug=True)


Connection successful
Dash app running on http://127.0.0.1:14811/
